In [1]:
# SVR GA FEATURE EXTRACTION AND MODELLING
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from ga_svr import GA_CV

In [2]:
file_name = 'data.xlsx'
name_x = 'X'
name_y = 'y'
## Split 1:X
x_df = pd.read_excel(file_name, sheet_name=name_x, index_col=0)
y_df = pd.read_excel(file_name, sheet_name=name_y, index_col=0)

selected_y = 'y'

In [3]:
split_type = input("Split wlasny (W) czy algorytmem 1 do X? (int np. 2,3,4...n) ")


In [4]:
if split_type.lower() == 'w':
    x_sorted = x_df.join(y_df[selected_y])
    nan_train = x_sorted.columns[x_sorted.isna().any()].tolist()
    print('---------------------')
    print('Training set NaNs: ', nan_train)

    train_x = x_sorted.drop(nan_train, axis=1)
    print('Checking again...')
    nan_train = train_x.columns[train_x.isna().any()].tolist()
    print('Training set NaNs: ', nan_train)
    
    y_sorted = x_sorted[selected_y]
    train_y = y_sorted.copy()
    Xt = train_x[train_x['split']=='T']
    Yt = y_sorted[train_x['split']=='T']
    Xv = train_x[train_x['split']=='V']
    Yv = y_sorted[train_x['split']=='V']
    Xt = Xt.drop(['split', selected_y], axis=1)
    Xv = Xv.drop(['split', selected_y], axis=1)

    from sklearn.feature_selection import VarianceThreshold
    selector = VarianceThreshold(threshold=0.0)
    selector.fit(Xt)
    indexes = selector.get_support()
    train_x = Xt.iloc[:,indexes]
    val_x = Xv.iloc[:,indexes]
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    Xt = pd.DataFrame(scaler.fit_transform(train_x), 
                       columns=train_x.columns.values, 
                       index=train_x.index.values)
    Xv = pd.DataFrame(scaler.transform(val_x), 
                       columns=val_x.columns.values, 
                       index=val_x.index.values)
else:
    x_sorted = x_df.join(y_df[selected_y]).sort_values(by=selected_y, ascending=True)
    y_sorted = x_sorted[selected_y]
    x_sorted.drop(selected_y, axis=1, inplace=True)
    if len([x for x in x_sorted.columns.values if x == 'split']) > 0:
        x_sorted.drop('split', axis=1, inplace=True)
    xdon = int(split_type)
    nan_train = x_sorted.columns[x_sorted.isna().any()].tolist()

    print('---------------------')
    print('Training set NaNs: ', nan_train)

    train_x = x_sorted.drop(nan_train, axis=1)
    print('Checking again...')
    nan_train = train_x.columns[train_x.isna().any()].tolist()
    print('Training set NaNs: ', nan_train)

    #usuwa zmienne o constant wartosciach
    from sklearn.feature_selection import VarianceThreshold
    selector = VarianceThreshold()
    cut = selector.fit_transform(train_x)
    indexes = selector.get_support()
    train_x = train_x.iloc[:,indexes]
    train_y = y_sorted.copy()

    def x_to_n(X,n=3):
        row_val = X.shape[0]
        train_split = []
        val_split = []  
        for row in range(0, row_val):
            #print(row % n)
            if ((row%n) < n-1):
                train_split.append(row)
            else:
                val_split.append(row)
        last_train = np.max(train_split)
        last_val = np.max(val_split)
        if (np.max(train_split) < np.max(val_split)):
            train_split.pop()
            val_split.pop()
            train_split.append(last_val)
            val_split.append(last_train)
            
        return train_split, val_split

    ts, vs = x_to_n(train_x, xdon)

    Xt = train_x.iloc[ts]
    Xv = train_x.iloc[vs]
    Yt = train_y.iloc[ts]
    Yv = train_y.iloc[vs]
    
    col_names = Xt.columns.values
    tr_sample_names = Xt.index.values
    v_sample_names = Xv.index.values
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(Xt)
    Xt = pd.DataFrame(scaler.transform(Xt), index = tr_sample_names, columns=col_names)
    Xv = pd.DataFrame(scaler.transform(Xv), index = v_sample_names, columns=col_names)



---------------------
Training set NaNs:  []
Checking again...
Training set NaNs:  []


In [5]:
X = Xt
Y = Yt
X_names = Xt.columns.values

is_array=False
try:
    X = np.array(X)
    is_array = True
except:
    X = X
    
isy_array=False
try:
    Y = np.array(Y)
    isy_array = True
except:
    Y = Y

In [6]:
n_vars = 2          
iters = 10         
mut_rat = 0.05       
pop_size = 1000      

eps = 0.1           
kernel = 'rbf'   
C_val = 1           
degree = 3          

In [7]:
model_vars = ['VE2sign_Dz(e)', 'ATS3i']

temp_X = Xt[model_vars]
temp_V = Xv[model_vars]

selected_X = temp_X 
selected_V = temp_V 

svr = SVR(epsilon=eps, kernel=kernel, C=C_val, degree=degree, max_iter=-1)
svr.fit(selected_X, Yt)
train_pred = svr.predict(selected_X)
test_pred = svr.predict(selected_V)

train_obs = np.asarray(Yt).reshape(-1)
train_pred = np.asarray(train_pred).reshape(-1)
test_obs = np.asarray(Yv).reshape(-1)
test_pred = np.asarray(test_pred).reshape(-1)


rmse_of_cal = np.sqrt(metrics.mean_squared_error(Yt, train_pred))
rmse_of_val = np.sqrt(metrics.mean_squared_error(Yv, test_pred))
mae_c = metrics.mean_absolute_error(Yt, train_pred)
mae_v = metrics.mean_absolute_error(Yv, test_pred)


r2= svr.score(selected_X, Yt)
q2= svr.score(selected_V, Yv)

print("R2 modelu", round(r2,3))
print("Q2 modelu", round(q2,3))
print("RMSEc modelu", round(rmse_of_cal,3))
print("RMSEv modelu", round(rmse_of_val,3))
print("MAE_c", round(mae_c, 3))
print("MAE_v", round(mae_v, 3))

R2 modelu 0.834
Q2 modelu 0.816
RMSEc modelu 0.15
RMSEv modelu 0.123
MAE_c 0.137
MAE_v 0.089


In [8]:
import plotly.graph_objs as go

Descriptor1_train = selected_X.iloc[:, 0].to_numpy()  
Descriptor2_train = selected_X.iloc[:, 1].to_numpy()
Descriptor1_test = selected_V.iloc[:, 0].to_numpy()
Descriptor2_test = selected_V.iloc[:, 1].to_numpy()

x_min, x_max = np.min(Descriptor1_train), np.max(Descriptor1_train)
y_min, y_max = np.min(Descriptor2_train), np.max(Descriptor2_train)
x_range = np.linspace(x_min, x_max, 50)
y_range = np.linspace(y_min, y_max, 50)
X, Y = np.meshgrid(x_range, y_range)
Z_train = svr.predict(np.c_[X.ravel(), Y.ravel()]).reshape(X.shape)

fig = go.Figure(data=[
    go.Scatter3d(x=Descriptor1_train, y=Descriptor2_train, z=Yt, 
                 mode='markers', marker=dict(size=5, color='blue'), name='Training Set'),
    go.Scatter3d(x=Descriptor1_test, y=Descriptor2_test, z=Yv, 
                 mode='markers', marker=dict(size=5, color='red'), name='Validation Set'),
    go.Surface(x=X, y=Y, z=Z_train, colorscale='Viridis', opacity=0.7, name='Prediction Surface')
])

fig.update_layout(scene=dict(
                    xaxis_title='Descriptor 1',
                    yaxis_title='Descriptor 2',
                    zaxis_title='Predicted Y'),
                  title='SVR for Target Property',
                  width=900, height=700)

fig.show()

/Users/michalina/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(


SHAP

In [9]:

import shap

Xt['ID'] = list(range(1, len(Xt) + 1))
Xv['ID'] = list(range(len(Xt) + 1, len(Xt) + len(Xv) + 1))

temp_Xt = Xt[model_vars].values
temp_Xv = Xv[model_vars].values

combined_X = np.vstack((temp_Xt, temp_Xv))

explainer = shap.KernelExplainer(svr.predict, temp_Xt)

shap_values = explainer.shap_values(combined_X)

shap_values_df = pd.DataFrame(shap_values, columns=model_vars)
combined_X_df = pd.DataFrame(combined_X, columns=model_vars)

combined_X_df['ID'] = Xt['ID'].tolist() + Xv['ID'].tolist()

summary_df = pd.DataFrame()

for i, var in enumerate(model_vars):
    temp_df = pd.DataFrame()
    temp_df['Feature Value'] = combined_X_df[var]
    temp_df['SHAP Value'] = shap_values_df[var]
    temp_df['Feature'] = var
    temp_df['Feature Position'] = i 
    temp_df['ID'] = combined_X_df['ID']  
    summary_df = pd.concat([summary_df, temp_df], axis=0)

min_feature_value = combined_X_df[model_vars].min().min()
max_feature_value = combined_X_df[model_vars].max().max()

fig1 = go.Figure()
fig2 = go.Figure()

for i, feature in enumerate(model_vars):
    feature_data = summary_df[summary_df['Feature'] == feature]
    y_values = np.linspace(i - 0.5, i + 0.5, len(feature_data)) 

    if feature == 'VE2sign_Dz(e)':
        fig = fig1
    elif feature == 'ATS3i':
        fig = fig2

    fig.add_trace(go.Scatter(
        x=feature_data['SHAP Value'],
        y=y_values,
        mode='markers+text', 
        marker=dict(
            size=5,  
            symbol='circle', 
            color=feature_data['Feature Value'],
            colorscale='RdBu',
            showscale=True,
            colorbar=dict(title='Feature Value', tickformat='.2f'),  
            line=dict(width=1, color='black') 
        ),
        text=feature_data['ID'].astype(str),
        textposition='top center', 
        hoverinfo='text',
        name=feature,
        textfont=dict(size=8)
    ))

fig1.update_layout(
    title='SHAP Summary Plot - VE2sign_Dz(e)',
    xaxis_title='SHAP Value',
    yaxis_title='Feature',
    showlegend=False, 
    height=300,  
    hovermode='closest', 
    margin=dict(l=100, r=100, b=50, t=80),  
    font=dict(size=12),  
    yaxis=dict(range=[-0.5, 0.5], tickvals=[0], ticktext=['VE2sign_Dz(e)']),  
    xaxis=dict(showgrid=False), 
)

fig2.update_layout(
    title='SHAP Summary Plot - ATS3i',
    xaxis_title='SHAP Value',
    yaxis_title='Feature',
    showlegend=False,  
    height=300, 
    hovermode='closest', 
    margin=dict(l=100, r=100, b=50, t=80), 
    font=dict(size=12), 
    yaxis=dict(range=[0.5, 1.5], tickvals=[1], ticktext=['ATS3i']),  
    xaxis=dict(showgrid=False),  
)


fig1.show()
fig2.show()


  0%|          | 0/19 [00:00<?, ?it/s]

Predictions

In [10]:
df_x_to_pred = pd.read_excel('to_pred_reg.xlsx', index_col=0)

In [11]:
df_x_to_pred

,VE2sign_Dz(e),ATS3i
ID,,
1,0.003071,4.313430
2,0.000617,4.330059
11,0.004255,4.279374
67,0.006057,3.438032
82,0.001280,4.982196
...,...,...
12635,0.000250,5.010229
12636,0.000736,4.417346
12637,0.001388,4.936330


In [12]:
model_vars = ['VE2sign_Dz(e)',	'ATS3i'] 
df_x_to_pred = df_x_to_pred[model_vars]
df_x_to_pred = df_x_to_pred.dropna(axis=0)

In [13]:
df_x_to_pred.dropna(inplace=True)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df_x_to_pred) 

df_x_to_pred_scaled = scaler.transform(df_x_to_pred)
df_x_to_pred_scaled = pd.DataFrame(df_x_to_pred_scaled, columns=model_vars, index=df_x_to_pred.index)


In [15]:
predictions = svr.predict(df_x_to_pred_scaled)


In [16]:
df_przewidywanki = pd.DataFrame(predictions, 
                                columns=['New_predictions'], 
                                index=df_x_to_pred.index.values)

In [17]:
df_przewidywanki

,New_predictions
1,-0.049598
2,-0.047281
11,-0.089972
67,-0.068794
82,0.183989
...,...
12635,0.194727
12636,-0.007906
12637,0.178165
12638,0.183076
